In [1]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Load tickerdata
tickerdata = pd.read_excel(r'Data for Thesis.xlsx',engine='openpyxl',dtype=str)
tickerdata.head()

,Sharing Economy Firm,t,Incumbent Firms,Ticker,Exchange,CI,Market Index,t-1,t+1,t-3,...,t+12,ISP (t),ISP (t-1),ISP (t+1),ISP (t-3),ISP (t+3),ISP (t-6),ISP (t+6),ISP (t-12),ISP (t+12)
0,Airbnb,2008-08-31 00:00:00,Marriott International,MAR,NASDAQ,1,IXIC,2008-07-31 00:00:00,2008-09-30 00:00:00,2008-05-31 00:00:00,...,2009-08-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Airbnb,2008-08-31 00:00:00,Las Vegas Sands,LVS,NYSE,0,NYA,2008-07-31 00:00:00,2008-09-30 00:00:00,2008-05-31 00:00:00,...,2009-08-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bird,2017-09-30 00:00:00,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,2017-08-30 00:00:00,2017-10-30 00:00:00,2017-06-30 00:00:00,...,2018-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bird,2017-09-30 00:00:00,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,2017-08-30 00:00:00,2017-10-30 00:00:00,2017-06-30 00:00:00,...,2018-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Boatsetter,2012-10-31 00:00:00,Brunswick Corp,BC,NYSE,1,NYA,2012-09-30 00:00:00,2012-11-30 00:00:00,2012-07-31 00:00:00,...,2013-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Reshape/melt ticker data to make time an attribute
transformedtickerdata = tickerdata.melt(id_vars=['Sharing Economy Firm', 'Incumbent Firms', 'Ticker',
       'Exchange', 'CI', 'Market Index', 'ISP (t)', 'ISP (t-1)', 'ISP (t+1)', 'ISP (t-3)', 'ISP (t+3)',
       'ISP (t-6)', 'ISP (t+6)', 'ISP (t-12)', 'ISP (t+12)'],var_name='t',value_name='Date').drop(['ISP (t-1)', 'ISP (t+1)', 'ISP (t-3)', 'ISP (t+3)', 'ISP (t-6)', 'ISP (t+6)', 'ISP (t-12)', 'ISP (t+12)'],1)
transformedtickerdata['Ticker'] = transformedtickerdata['Ticker'].astype(str)
transformedtickerdata.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISP (t),t,Date
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,NaN,t,2008-08-31 00:00:00
1,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,NaN,t,2008-08-31 00:00:00
2,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,NaN,t,2017-09-30 00:00:00
3,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,NaN,t,2017-09-30 00:00:00
4,Boatsetter,Brunswick Corp,BC,NYSE,1,NYA,NaN,t,2012-10-31 00:00:00


In [4]:
#Load equity data and transform
equitydata = pd.read_excel(r'Bloomberg Equity Data.xlsx', engine='openpyxl',usecols=[0,1,2],sheet_name=None,dtype=str)
datalist = []
for ticker, data in equitydata.items():
    data['Ticker'] = str(ticker)
    datalist.append(data)
transformedequitydata = pd.concat(datalist,axis=0)
transformedequitydata.dropna(axis=1,how='all')
transformedequitydata['Last Price'] = pd.to_numeric(transformedequitydata['Last Price'])
transformedequitydata.head()

,Date,Last Price,Volume,Ticker
0,2009-08-31 00:00:00,22.4513,3111839,MAR
1,2009-08-30 00:00:00,23.0337,4564532,MAR
2,2009-08-29 00:00:00,23.0337,4564532,MAR
3,2009-08-28 00:00:00,23.0337,4564532,MAR
4,2009-08-27 00:00:00,22.9022,4047335,MAR


In [5]:
#Load index data and transform
# indexdata = pd.read_excel(r'Bloomberg Index Data.xlsx', engine='openpyxl',usecols=[0,1],sheet_name=None,dtype=str)
# indexlist = []
# for index, data in indexdata.items():
#     data['Index'] = str(index)
#     indexlist.append(data)
# transformedindexdata = pd.concat(indexlist,axis=0)
# transformedindexdata['Last Price'] = pd.to_numeric(transformedindexdata['Last Price'])
# transformedindexdata.head()

In [6]:
#Merge Ticker and Equity Datasets
merged_data = transformedtickerdata.merge(transformedequitydata, how='left',left_on=['Date','Ticker'],right_on=['Date','Ticker']).dropna(axis=1,how='all').rename(columns={'Last Price':'ISP (t)'})
merged_data['Volume'] = pd.to_numeric(merged_data['Volume'])
merged_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,t,Date,ISP (t),Volume
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,t,2008-08-31 00:00:00,26.30290,2756931
1,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,t,2008-08-31 00:00:00,47.41000,3567702
2,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,t,2017-09-30 00:00:00,4.72181,32850
3,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,t,2017-09-30 00:00:00,43.10000,316880
4,Boatsetter,Brunswick Corp,BC,NYSE,1,NYA,t,2012-10-31 00:00:00,23.59000,1002934


In [7]:
#Merge Index Data with Rest of Dataset
# merged_data=merged_data.merge(transformedindexdata,how='left',left_on=['Date','Market Index'],right_on=['Date','Index']).drop(columns='Index').rename(columns={'Last Price':'Index Level'})
# merged_data.head()

In [8]:
#Re-Pivot Data
complete_data = merged_data.pivot(index=['Sharing Economy Firm','Incumbent Firms','Ticker','Exchange','CI','Market Index'],columns=['t'])
complete_data.head()

Date  \
t                                                                                                  t   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2008-08-31 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2008-08-31 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2017-09-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2017-09-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2012-10-31 00:00:00   

                                                                                                      \
t                                                                                                t+1   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2008-09-30 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2008-09-30 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2017-10-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2017-10-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2012-11-30 00:00:00   

                                                                                                      \
t                                                                                               t+12   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2009-08-31 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2009-08-31 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2018-09-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2018-09-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2013-10-31 00:00:00   

                                                                                                      \
t                                                                                                t+3   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2008-11-30 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2008-11-30 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2017-12-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2017-12-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2013-01-31 00:00:00   

                                                                                                      \
t                                                                                                t+6   
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2009-02-28 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2009-02-28 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2018-03-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2018-03-30 00:00:00   
Boatsetter           Brunswick Corp 

In [9]:
#Drop Multindex
complete_data.columns = ['_'.join(col) for col in complete_data.columns]
complete_data.head()

Date_t  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2008-08-31 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2008-08-31 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2017-09-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2017-09-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2012-10-31 00:00:00   

                                                                                            Date_t+1  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2008-09-30 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2008-09-30 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2017-10-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2017-10-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2012-11-30 00:00:00   

                                                                                           Date_t+12  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2009-08-31 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2009-08-31 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2018-09-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2018-09-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2013-10-31 00:00:00   

                                                                                            Date_t+3  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2008-11-30 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2008-11-30 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2017-12-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2017-12-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2013-01-31 00:00:00   

                                                                                            Date_t+6  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2009-02-28 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2009-02-28 00:00:00   
Bird                 Fox Factory Holding Corp   FOXF   NASDAQ   0  IXIC          2018-03-30 00:00:00   
                     Giant Manufacturing Co Ltd 9921   TWSE     1  TAIEX         2018-03-30 00:00:00   
Boatsetter           Brunswick Corp             BC     NYSE     1  NYA           2013-04-30 00:00:00   

                                                                                            Date_t-1  \
Sharing Economy Firm Incumbent Firms            Ticker Exchange CI Market Index                        
Airbnb               Las Vegas Sands            LVS    NYSE     0  NYA           2008-07-31 00:00:00   
                     Marriott International     MAR    NASDAQ   1  IXIC          2008-07-31 00:00:00   
Bird                 Fox Factory 

In [10]:
#Drop Date Columns/Reset Index.
complete_data = complete_data.drop(columns=['Date_t', 'Date_t+1','Date_t+12','Date_t+3','Date_t+6','Date_t-1','Date_t-12','Date_t-3','Date_t-6']).reset_index().rename(columns={'ISP (t)_t':'ISP(t)','ISP (t)_t+1':'ISP(t+1)','ISP (t)_t+12':'ISP(t+12)','ISP (t)_t+3':'ISP(t+3)','ISP (t)_t+6':'ISP(t+6)','ISP (t)_t-1':'ISP(t-1)','ISP (t)_t-12':'ISP(t-12)','ISP (t)_t-3':'ISP(t-3)','ISP (t)_t-6':'ISP(t-6)','Volume_t':'Volume(t)', 'Volume_t+1':'Volume(t+1)', 'Volume_t+12':'Volume(t+12)', 'Volume_t+3':'Volume(t+3)', 'Volume_t+6':'Volume(t+6)', 'Volume_t-1':'Volume(t-1)', 'Volume_t-12':'Volume(t-12)', 'Volume_t-3':'Volume(t-3)', 'Volume_t-6':'Volume(t-6)'})
# , 'Index Level_t':'Index Level(t)', 'Index Level_t+1':'Index Level(t+1)','Index Level_t+12':'Index Level(t+12)', 'Index Level_t+3':'Index Level(t+3)', 'Index Level_t+6':'Index Level(t+6)','Index Level_t-1':'Index Level(t-1)', 'Index Level_t-12':'Index Level(t-12)', 'Index Level_t-3':'Index Level(t-3)','Index Level_t-6':'Index Level(t-6)'
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISP(t),ISP(t+1),ISP(t+12),ISP(t+3),...,ISP(t-6),Volume(t),Volume(t+1),Volume(t+12),Volume(t+3),Volume(t+6),Volume(t-1),Volume(t-12),Volume(t-3),Volume(t-6)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,47.41000,36.11000,14.2600,5.1700,...,83.30000,3567702,2854762,22988568,12411226,28421998,5017749,2489400,2883924,2853416
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,26.30290,24.32620,22.4513,15.6549,...,31.79470,2756931,8810031,3111839,1496529,6908523,3808057,1691184,3861316,3165477
2,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,43.10000,42.95000,70.0500,38.8500,...,28.60000,316880,200100,301258,216524,449896,332831,342422,412887,372828
3,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,4.72181,5.10509,4.2972,5.5108,...,5.90915,32850,177225,616420,493016,288304,1008183,558079,647519,2191899
4,Boatsetter,Brunswick Corp,BC,NYSE,1,NYA,23.59000,25.77000,45.1300,36.1600,...,26.29000,1002934,678508,1118859,824993,1454809,1204379,2184608,829653,1385213


In [11]:
# #Add Columns for Index Level Growth Rate % for t=((+-)1,3,6,12)
# complete_data['% Change in Index Level (t-1)'] = ((complete_data['Index Level(t)']-complete_data['Index Level(t-1)'])/complete_data['Index Level(t-1)'])*100
# complete_data['% Change in Index Level (t+1)'] = ((complete_data['Index Level(t+1)']-complete_data['Index Level(t)'])/complete_data['Index Level(t)'])*100
# complete_data['% Change in Index Level (t-3)'] = ((complete_data['Index Level(t)']-complete_data['Index Level(t-3)'])/complete_data['Index Level(t-3)'])*100
# complete_data['% Change in Index Level (t+3)'] = ((complete_data['Index Level(t+3)']-complete_data['Index Level(t)'])/complete_data['Index Level(t)'])*100
# complete_data['% Change in Index Level (t-6)'] = ((complete_data['Index Level(t)']-complete_data['Index Level(t-6)'])/complete_data['Index Level(t-6)'])*100
# complete_data['% Change in Index Level (t+6)'] = ((complete_data['Index Level(t+6)']-complete_data['Index Level(t)'])/complete_data['Index Level(t)'])*100
# complete_data['% Change in Index Level (t-12)'] = ((complete_data['Index Level(t)']-complete_data['Index Level(t-12)'])/complete_data['Index Level(t-12)'])*100
# complete_data['% Change in Index Level (t+12)'] = ((complete_data['Index Level(t+12)']-complete_data['Index Level(t)'])/complete_data['Index Level(t)'])*100
# complete_data.head()

In [12]:
#Add Columns for Volume Growth Rate % for t=((+-)1,3,6,12)
complete_data['% Change in Volume (t-1)'] = ((complete_data['Volume(t)']-complete_data['Volume(t-1)'])/complete_data['Volume(t-1)'])*100
complete_data['% Change in Volume (t+1)'] = ((complete_data['Volume(t+1)']-complete_data['Volume(t)'])/complete_data['Volume(t)'])*100
complete_data['% Change in Volume (t-3)'] = ((complete_data['Volume(t)']-complete_data['Volume(t-3)'])/complete_data['Volume(t-3)'])*100
complete_data['% Change in Volume (t+3)'] = ((complete_data['Volume(t+3)']-complete_data['Volume(t)'])/complete_data['Volume(t)'])*100
complete_data['% Change in Volume (t-6)'] = ((complete_data['Volume(t)']-complete_data['Volume(t-6)'])/complete_data['Volume(t-6)'])*100
complete_data['% Change in Volume (t+6)'] = ((complete_data['Volume(t+6)']-complete_data['Volume(t)'])/complete_data['Volume(t)'])*100
complete_data['% Change in Volume (t-12)'] = ((complete_data['Volume(t)']-complete_data['Volume(t-12)'])/complete_data['Volume(t-12)'])*100
complete_data['% Change in Volume (t+12)'] = ((complete_data['Volume(t+12)']-complete_data['Volume(t)'])/complete_data['Volume(t)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISP(t),ISP(t+1),ISP(t+12),ISP(t+3),...,Volume(t-3),Volume(t-6),% Change in Volume (t-1),% Change in Volume (t+1),% Change in Volume (t-3),% Change in Volume (t+3),% Change in Volume (t-6),% Change in Volume (t+6),% Change in Volume (t-12),% Change in Volume (t+12)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,47.41000,36.11000,14.2600,5.1700,...,2883924,2853416,-28.898357,-19.983171,23.709987,247.877317,25.032663,696.647198,43.315739,544.352247
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,26.30290,24.32620,22.4513,15.6549,...,3861316,3165477,-27.602686,219.559358,-28.601259,-45.717575,-12.906301,150.587447,63.017803,12.873300
2,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,43.10000,42.95000,70.0500,38.8500,...,412887,372828,-4.792522,-36.853067,-23.252609,-31.670033,-15.006384,41.976774,-7.459217,-4.929942
3,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,4.72181,5.10509,4.2972,5.5108,...,647519,2191899,-96.741663,439.497717,-94.926790,1400.809741,-98.501300,777.637747,-94.113737,1776.468798
4,Boatsetter,Brunswick Corp,BC,NYSE,1,NYA,23.59000,25.77000,45.1300,36.1600,...,829653,1385213,-16.726047,-32.347692,20.885961,-17.742045,-27.597128,45.055308,-54.090894,11.558587


In [13]:
#Add Columns for Share Price Growth Rate % for t=((+-)1,3,6,12)
complete_data['% Change in Share Price (t-1)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-1)'])/complete_data['ISP(t-1)'])*100
complete_data['% Change in Share Price (t+1)'] = ((complete_data['ISP(t+1)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100
complete_data['% Change in Share Price (t-3)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-3)'])/complete_data['ISP(t-3)'])*100
complete_data['% Change in Share Price (t+3)'] = ((complete_data['ISP(t+3)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100
complete_data['% Change in Share Price (t-6)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-6)'])/complete_data['ISP(t-6)'])*100
complete_data['% Change in Share Price (t+6)'] = ((complete_data['ISP(t+6)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100
complete_data['% Change in Share Price (t-12)'] = ((complete_data['ISP(t)']-complete_data['ISP(t-12)'])/complete_data['ISP(t-12)'])*100
complete_data['% Change in Share Price (t+12)'] = ((complete_data['ISP(t+12)']-complete_data['ISP(t)'])/complete_data['ISP(t)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISP(t),ISP(t+1),ISP(t+12),ISP(t+3),...,% Change in Volume (t-12),% Change in Volume (t+12),% Change in Share Price (t-1),% Change in Share Price (t+1),% Change in Share Price (t-3),% Change in Share Price (t+3),% Change in Share Price (t-6),% Change in Share Price (t+6),% Change in Share Price (t-12),% Change in Share Price (t+12)
0,Airbnb,Las Vegas Sands,LVS,NYSE,0,NYA,47.41000,36.11000,14.2600,5.1700,...,43.315739,544.352247,4.152021,-23.834634,-31.725230,-89.095128,-43.085234,-95.190888,-52.447342,-69.921957
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,26.30290,24.32620,22.4513,15.6549,...,63.017803,12.873300,8.876830,-7.515141,-14.281478,-40.482228,-17.272690,-49.805155,-36.492656,-14.643252
2,Bird,Fox Factory Holding Corp,FOXF,NASDAQ,0,IXIC,43.10000,42.95000,70.0500,38.8500,...,-7.459217,-4.929942,8.291457,-0.348028,21.067416,-9.860789,50.699301,-19.025522,87.636047,62.529002
3,Bird,Giant Manufacturing Co Ltd,9921,TWSE,1,TAIEX,4.72181,5.10509,4.2972,5.5108,...,-94.113737,1776.468798,-7.085187,8.117226,-17.569188,16.709482,-20.093245,12.054699,-33.146868,-8.992526
4,Boatsetter,Brunswick Corp,BC,NYSE,1,NYA,23.59000,25.77000,45.1300,36.1600,...,-54.090894,11.558587,4.242156,9.241204,7.276035,53.285290,-10.270065,34.209411,33.578709,91.309877


In [ ]:
#Create Function to turn 0% Growth Rates to .000001% so that Python does not return infinite values for % change in Growth Rates
def make_non_zero(df, columns):
    for column in columns:
        df.loc[df[column] == 0, column] = .000001
make_non_zero(complete_data,complete_data.columns[24:39])
complete_data.head()

In [ ]:
# #Add Columns for % Change in Growth Rate of Index Level
# complete_data['% Change in Growth Rate of Index Level (t=1)'] = ((complete_data['% Change in Index Level (t+1)']-complete_data['% Change in Index Level (t-1)'])/complete_data['% Change in Index Level (t-1)'])*100
# complete_data['% Change in Growth Rate of Index Level (t=3)'] = ((complete_data['% Change in Index Level (t+3)']-complete_data['% Change in Index Level (t-3)'])/complete_data['% Change in Index Level (t-3)'])*100
# complete_data['% Change in Growth Rate of Index Level (t=6)'] = ((complete_data['% Change in Index Level (t+6)']-complete_data['% Change in Index Level (t-6)'])/complete_data['% Change in Index Level (t-6)'])*100
# complete_data['% Change in Growth Rate of Index Level (t=12)'] = ((complete_data['% Change in Index Level (t+12)']-complete_data['% Change in Index Level (t-12)'])/complete_data['% Change in Index Level (t-12)'])*100
# complete_data.head()

In [ ]:
#Add Columns for % Change in Growth Rate of Volume
complete_data['% Change in Growth Rate of Volume (t=1)'] = ((complete_data['% Change in Volume (t+1)']-complete_data['% Change in Volume (t-1)'])/complete_data['% Change in Volume (t-1)'])*100
complete_data['% Change in Growth Rate of Volume (t=3)'] = ((complete_data['% Change in Volume (t+3)']-complete_data['% Change in Volume (t-3)'])/complete_data['% Change in Volume (t-3)'])*100
complete_data['% Change in Growth Rate of Volume (t=6)'] = ((complete_data['% Change in Volume (t+6)']-complete_data['% Change in Volume (t-6)'])/complete_data['% Change in Volume (t-6)'])*100
complete_data['% Change in Growth Rate of Volume (t=12)'] = ((complete_data['% Change in Volume (t+12)']-complete_data['% Change in Volume (t-12)'])/complete_data['% Change in Volume (t-12)'])*100
complete_data.head()

In [ ]:
#Add Columns for % Change in Growth Rate of Share Price
complete_data['% Change in Growth Rate of Share Price (t=1)'] = ((complete_data['% Change in Share Price (t+1)']-complete_data['% Change in Share Price (t-1)'])/complete_data['% Change in Share Price (t-1)'])*100
complete_data['% Change in Growth Rate of Share Price (t=3)'] = ((complete_data['% Change in Share Price (t+3)']-complete_data['% Change in Share Price (t-3)'])/complete_data['% Change in Share Price (t-3)'])*100
complete_data['% Change in Growth Rate of Share Price (t=6)'] = ((complete_data['% Change in Share Price (t+6)']-complete_data['% Change in Share Price (t-6)'])/complete_data['% Change in Share Price (t-6)'])*100
complete_data['% Change in Growth Rate of Share Price (t=12)'] = ((complete_data['% Change in Share Price (t+12)']-complete_data['% Change in Share Price (t-12)'])/complete_data['% Change in Share Price (t-12)'])*100
complete_data.head()

In [ ]:
#Convert Exchange to Dummy Variables for either NYSE, NASDAQ, or Other (OTCMKTS, TWSE, and NSE)
exchange_dummies = pd.get_dummies(complete_data['Exchange'])
complete_data = pd.concat([complete_data,exchange_dummies],axis=1)
complete_data['Other'] = complete_data['NSE']+complete_data['OTCMKTS']+complete_data['TWSE']
complete_data = complete_data.drop(columns=['NSE','OTCMKTS','TWSE']).sort_values(by=['Sharing Economy Firm','CI'],ascending=[True, False])
complete_data.head()

In [ ]:
#Write complete data to Excel
with pd.ExcelWriter('complete_data.xlsx',engine='openpyxl') as writer:
    complete_data.to_excel(writer)

In [ ]:
#Create STATA ready data
stata_data = complete_data[['CI', 'NASDAQ', 'NYSE', 'Other', '% Change in Growth Rate of Volume (t=1)', '% Change in Growth Rate of Volume (t=3)','% Change in Growth Rate of Volume (t=6)','% Change in Growth Rate of Volume (t=12)', '% Change in Growth Rate of Share Price (t=1)', '% Change in Growth Rate of Share Price (t=3)', '% Change in Growth Rate of Share Price (t=6)', '% Change in Growth Rate of Share Price (t=12)']].rename(columns={'% Change in Growth Rate of Volume (t=1)':'one_m_diff_volume', '% Change in Growth Rate of Volume (t=3)':'three_m_diff_volume','% Change in Growth Rate of Volume (t=6)':'six_m_diff_volume','% Change in Growth Rate of Volume (t=12)':'one_y_diff_volume', '% Change in Growth Rate of Share Price (t=1)':'one_m_diff_shareprice', '% Change in Growth Rate of Share Price (t=3)':'three_m_diff_shareprice', '% Change in Growth Rate of Share Price (t=6)':'six_m_diff_shareprice', '% Change in Growth Rate of Share Price (t=12)':'one_y_diff_shareprice'}).set_index('CI')
# '% Change in Growth Rate of Index Level (t=1)', '% Change in Growth Rate of Index Level (t=3)','% Change in Growth Rate of Index Level (t=6)', '% Change in Growth Rate of Index Level (t=12)',
# '% Change in Growth Rate of Index Level (t=1)':'one_m_diff_indexlevel', '% Change in Growth Rate of Index Level (t=3)':'three_m_diff_indexlevel','% Change in Growth Rate of Index Level (t=6)':'six_m_diff_indexlevel', '% Change in Growth Rate of Index Level (t=12)':'one_y_diff_indexlevel',
stata_data.head()

In [ ]:
#Write STATA data to Excel and CSV
with pd.ExcelWriter('stata_data.xlsx',engine='openpyxl') as writer:
    stata_data.to_excel(writer)
stata_data.to_csv('stata_data.csv')